In [ ]:
import time
import matplotlib.pyplot as plt
import numpy as np
import mxnet as mx
from model import SimpleStack
from utils import check_dir
from memory import Memory
from algorithm.DQN import DQN
from environments.SimpleEnv import SimpleEnv
from utils import copy_params

In [ ]:
# agent view
agent_view = 7
map_size = 10
# action max
action_max = 3
# learning rate
model_save = "./model_save/"
lr = 0.002
# start play
replay_start_size = 20000
# update step
update_step = 1000
# gamma in q-loss calculation
gamma = 0.99
# memory pool size
memory_length = 500000
# file to save train log
summary = "./test_{}".format(str(time.time()))
# the number of step it take to linearly anneal the epsilon to it min value
annealing_end = 1000000
# min level of stochastically of policy (epsilon)-greedy
epsilon_min = 0.2
# temporary files
temporary_model = "./{}/model.params".format(model_save)
temporary_pool = "./{}/pool".format(model_save)

In [ ]:
import os
if os.path.exists(summary):
    os.remove(summary)

In [ ]:
ctx = mx.gpu()
for i in ["model_save", "data_save"]:
    check_dir(i)
# build models
online_model = SimpleStack(agent_view, map_size)
offline_model = SimpleStack(agent_view, map_size)
online_model.collect_params().initialize(mx.init.Normal(0.02), ctx=ctx)
offline_model.collect_params().initialize(mx.init.Normal(0.02), ctx=ctx)
offline_model.collect_params().zero_grad()
print("create model")

In [ ]:
online_model

In [ ]:
# load_model = "./phase1.params"
# if os.path.exists(temporary_model):
#     online_model.load_parameters(load_model, ctx=ctx)
#     offline_model.load_parameters(load_model, ctx=ctx)
#     print("load model")

In [ ]:
env = SimpleEnv(display=False)
env.reset_env()
# create pool
memory_pool = Memory(memory_length)
algorithm = DQN([online_model, offline_model], ctx, lr, gamma, memory_pool, action_max, temporary_model,bz=1024)

In [ ]:
finish = 0
all_step_counter = 0
annealing_count = 0
cost = []
texts = []
num_episode = 500000
tot_reward = np.zeros(num_episode)
moving_average_clipped = 0.
moving_average = 0.

In [ ]:
_epoch = 0

In [ ]:
negative_counter = 0
tmp_reward = 0
for epoch in range(_epoch, num_episode):
    _epoch += 1
    env.reset_env()
    finish = 0
    cum_clipped_reward = 0
    while not finish:
        if all_step_counter > replay_start_size:
            annealing_count += 1
        if all_step_counter == replay_start_size:
            print('annealing and learning are started')
        eps = np.maximum(1 - all_step_counter / annealing_end, epsilon_min)
        action, by = algorithm.get_action(env.state(), eps)
        old, new, reward_get, finish, original_reward = env.step(action)
        memory_pool.add(old, new, action, reward_get, finish)
        cum_clipped_reward += original_reward
        all_step_counter += 1
        if finish and len(env.finish) > 50:
            sr_50 = sum(env.finish[-50:]) / min(len(env.finish), 50)
            ar_50 = sum(env.total_reward[-50:]) / sum(env.total_step_count[-50:])
            sr_all = sum(env.finish) / len(env.finish)
            ar_all = sum(env.total_reward) / sum(env.total_step_count)
            text = "success rate last 50 %f, avg return %f; success rate total %f, avg return total %f" % (
                sr_50, ar_50, sr_all, ar_all)
            with open(summary, "a") as f:
                f.writelines(text + "\n")
            if epoch % 100 == 0:
                print(text + "; %f" % eps)
        # save model and replace online model each epoch
        if annealing_count > replay_start_size and annealing_count % update_step == 0:
            copy_params(offline_model, online_model)
            offline_model.save_parameters(temporary_model)
    #  train every 4 epoch
    if annealing_count > replay_start_size and epoch % 4 == 0:
        cost.append(algorithm.train())
    tot_reward[int(epoch) - 1] = cum_clipped_reward

In [ ]:
_number = "1"
bandwidth = 1000 # Moving average bandwidth
total_rew = np.zeros(int(_epoch)-bandwidth)
for i in range(int(_epoch)-bandwidth):
    total_rew[i] = np.sum(tot_reward[i:i+bandwidth])/bandwidth
t = np.arange(int(_epoch)-bandwidth)
belplt = plt.plot(t,total_rew[0:int(_epoch)-bandwidth],"r", label = "Return")
#handles[belplt])
plt.legend()
print('Running after %d number of episodes' %_epoch)
plt.xlabel("Number of episode")
plt.ylabel("Average Reward per episode")
plt.savefig("R5_train_%s.jpg" % _number)
plt.show()
np.save("R5_train_%s.array" % _number, env.total_step_count)